In [1]:
import time
import requests
from lxml import etree
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=options)
driver.get('https://www.google.com/search?q=celtic+park+glasgow+google+review') # Change URL here

time.sleep(1)

# Find and click on the "Read All Reviews" button
read_all_reviews_button = driver.find_element(By.XPATH, "//span[@class='hqzQac']/span/a")
read_all_reviews_button.click()

# Wait for the pop-up box to load
popup_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class ='review-dialog-list']")))

data = []
unique_reviews = set()

# Function to extract review data
def extract_reviews():
    reviews = popup_box.find_elements(By.XPATH, "//div[contains(@class,'WMbnJf vY6njf gws-localreviews__google-review')]")
    for review in reviews:
        try:
            name = review.find_element(By.XPATH, ".//div[contains(@class,'TSUbDb')]/a").text
            date = review.find_element(By.XPATH, ".//span[contains(@class,'dehysf lTi8oc')]").text
            customer_review = review.find_element(By.XPATH, ".//span[contains(@class,'review-snippet')]").text
            
            # Create a unique identifier for each review
            review_id = (name, date, customer_review)
            
            if review_id not in unique_reviews:
                unique_reviews.add(review_id)
                data.append([name, date, customer_review])
        except:
            continue
    return len(reviews)

# Scroll within the pop-up box to load more reviews
while True:
    previous_reviews_count = len(data)
    # Scroll down within the pop-up box by a larger height
    
    driver.execute_script("arguments[0].scrollBy(0, arguments[0].scrollHeight);", popup_box)
    time.sleep(1)
    extract_reviews()
    current_reviews_count = len(data)

    if current_reviews_count == previous_reviews_count:
        break  

driver.quit()
df = pd.DataFrame(data, columns=['Name', 'Date', 'Review'])
df

              Name   Date                                             Review
0      mete atasoy   1 週前                                                   
1      Steven Dunn  1 個月前  我已經說過這太棒了，我自己、我 11 歲的兒子，還有我可能的女婿，都喜歡它。 😁。\n對於任...
2  Maurice Kempers  4 個月前  球場非常棒，從某些角度來看屋頂的柱子有點煩人，但為俱樂部歡呼的人群會讓這一切都忘記。\n最好...
3   Rafique Gardee  2 個月前      獲得了一場精彩的主場勝利，這是一次很棒的體驗。\n\n球迷很友好，這裡也有很多家庭。 ⋯⋯
4      Awang Bagas  5 個月前  太棒了，這是我第一次觀看冠軍聯賽比賽！！！聽到倫敦大學學院的國歌就在我眼前響起，我起雞皮疙瘩...
5      Aaron Wiese  7 個月前                                                   
6       Bull Julia  8 個月前                                                   
7          cam jam  3 個月前                                                   
8       اولاد شريف   2 週前                                                   
